In [6]:
import psycopg2
import pandas as pd

def create_database():
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=1234")
    conn.set_session(autocommit=True)
    cur = conn.cursor()



    cur.execute("""
        SELECT pg_terminate_backend(pg_stat_activity.pid)
        FROM pg_stat_activity
        WHERE pg_stat_activity.datname = 'myfirst'
          AND pid <> pg_backend_pid();
    """)

    cur.execute("DROP DATABASE IF EXISTS myfirst")
    cur.execute("CREATE DATABASE myfirst")

    conn.close()

    conn = psycopg2.connect("host=localhost dbname=myfirst user=postgres password=1234")
    cur = conn.cursor()

    return cur, conn

def drop_tables(cur, conn):
    drop_tables_query = [
        "DROP TABLE IF EXISTS accountinformation"
    ]
    for query in drop_tables_query:
        cur.execute(query)
        conn.commit()

def create_tables(cur, conn):
    create_tables_query = """
    CREATE TABLE IF NOT EXISTS accountinformation (
        Loan_ID varchar,
        Gender varchar,
        Married varchar,
        Dependents varchar,
        Self_Employed varchar,
        ApplicantIncome int,
        CoapplicantIncome float,
        LoanAmount float,
        Credit_History float,
        Property_Area varchar,
        Loan_Status varchar
    )
    """
    cur.execute(create_tables_query)
    conn.commit()

def insert_data(cur, conn, dataframe):
    for index, row in dataframe.iterrows():
        cur.execute("""
        INSERT INTO accountinformation (
            Loan_ID, Gender, Married, Dependents, Self_Employed,
            ApplicantIncome, CoapplicantIncome, LoanAmount, 
            Credit_History, Property_Area, Loan_Status
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['Loan_ID'], row['Gender'], row['Married'], row['Dependents'], row['Self_Employed'],
            row['ApplicantIncome'], row['CoapplicantIncome'], row['LoanAmount'],
            row['Credit_History'], row['Property_Area'], row['Loan_Status']
        ))
    conn.commit()

# Load data from CSV into pandas DataFrame
finance = pd.read_csv("C:\\Users\\HASSAN\\Downloads\\train_ctrUa4K.csv")


# Clean data (selecting relevant columns)
finance_clean = finance[['Loan_ID', 'Gender', 'Married', 'Dependents', 'Self_Employed',
                         'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
                         'Credit_History', 'Property_Area', 'Loan_Status']]

# Execute database operations
cur, conn = create_database()
drop_tables(cur, conn)
create_tables(cur, conn)
insert_data(cur, conn, finance_clean)

# Close connections
cur.close()
conn.close()
